In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

data = pd.read_csv('wine.csv')
X1 = data.iloc[:,0:11]
Y = data.iloc[:,11:12].values

X1 = (X1 - X1.mean())/(X1.std())

X = np.ones([1599,1])
for column in X1:
    dummy=X1[column]
    dummy=pd.cut(dummy, bins=4, labels=False).values
    dummy=dummy.reshape(-1, 1)
    X = np.concatenate((X,dummy),axis=1)

for i in range(1599):
    if Y[i]<5:
        Y[i]=0
    elif Y[i]>6 :
        Y[i]=2
    else :
        Y[i]=1

X = np.concatenate((X,Y),axis=1)
X=X.astype(int)

In [2]:
def entropy(records,X):
    count1=count2=count0=0
    for i in records:
        if X[i,12]==0:
            count0=count0+1
        elif X[i,12]==1:
            count1=count1+1
        else :
            count2=count2+1
    total=count0+count1+count2
    if total:
        frac0=count0/total
        frac1=count1/total
        frac2=count2/total
        impurity=-(tlog(frac0)+tlog(frac1)+tlog(frac2))
    else:
        impurity=0
    return impurity
        

def split(records,maxatt,X):
    list1=[]
    list2=[]
    list3=[]
    list0=[]
    for i in records:
        if X[i,maxatt]==0:
            list0.append(i)
        elif X[i,maxatt]==1:
            list1.append(i)
        elif X[i,maxatt]==2:
            list2.append(i)
        else :
            list3.append(i)
        
    sep=[list0,list1,list2,list3]
    return sep

def tlog(x):
    if x:
        return x*(np.log2(x))
    else :
        return 0

def maxent(records,att,X):
    min=10000
    att1=90
    for i in att:
        temp=0
        sep=split(records,i,X)
        for j in sep:
            temp=temp+len(j)*(entropy(j,X))/len(records)
        if min > temp :
            min = temp
            att1=i
    return att1,min
        

class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)

In [9]:
def buildtree(records,att,X):
    if len(att)==0:
        count1=count2=count0=0
        for i in records:
            if X[i,12]==0:
                count0=count0+1
            elif X[i,12]==1:
                count1=count1+1
            else :
                count2=count2+1
        w=1
        if count0>count1:
            if count2>count0:
                w=2
            else:
                w=0
        else:
            if count2>count1:
                w=2
            else:
                w=1
        node=Node(w)   
        return node
    elif len(records)<10:
        count1=count2=count0=0
        for i in records:
            if X[i,12]==0:
                count0=count0+1
            elif X[i,12]==1:
                count1=count1+1
            else :
                count2=count2+1
        w=1
        if count0>count1:
            if count2>count0:
                w=2
            else:
                w=0
        else:
            if count2>count1:
                w=2
            else:
                w=1
        node=Node(w)    
        return node
    else:
        maxatt,maxentropy=maxent(records,att,X)
        sep=split(records,maxatt,X)
        node=Node(maxatt)
        if maxentropy==0:
            for i in range(4):
                if len(sep[i]) :
                    node1=Node(X[sep[i][0],12])   
                    node.add_child(node1)
                else :
                    node1=Node(1)   
                    node.add_child(node1) 
                
                
        else:
            att1 = att.copy()
            att1.remove(maxatt)
            for i in range(4):
                node.add_child(buildtree(sep[i],att1,X))
        return node

In [4]:
.
def predict(k,X,root):
        node=root
        while ((node.children!=[])):
            node=node.children[X[k,node.data]]
        return node.data

In [5]:
from numpy import array
from sklearn.model_selection import KFold
kfold = KFold(3, True, 2)

In [6]:
sum1=sum2=sum3=0
for train, test in kfold.split(X):
    X_train=X[train]
    X_test=X[test]
    a=np.arange(X_train.shape[0])
    b=np.arange(11)
    b=b+1
    records=list(a)
    att=list(b)
    root=buildtree(records,att,X_train)
    Y1 = np.ones([X_test.shape[0],1])
    for i in range(X_test.shape[0]):
        Y1[i,0]=predict(i,X_test,root)
    sum1=sum1+accuracy_score(X_test[:,12],Y1)
    sum2=sum2+precision_score(X_test[:,12],Y1,average='macro')
    sum3=sum3+recall_score(X_test[:,12],Y1,average='macro')
    

In [7]:
print('Accuracy = '+ str(sum1/3))
print('Precision = '+ str(sum2/3))
print('Recall = '+ str(sum3/3))

Accuracy = 0.8098811757348344
Precision = 0.5213945118529942
Recall = 0.44111360709657416
